In [2]:
import os
import json
import sys

from autogen import ConversableAgent
from bs4 import BeautifulSoup


# Define your model configuration
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# llm_config = {"model": "gpt-3.5-turbo"}
llm_config = {"model": "gpt-4o-mini"}
meta_html_jobs_dir = '/Users/vimal.chaudhari/Downloads/vimalKeshu/job-search/backend/crawler/meta/jobs'
meta_json_jobs_dir = '/Users/vimal.chaudhari/Downloads/vimalKeshu/job-search/backend/crawler/meta/jobs/json'
meta_job_url = "https://www.metacareers.com/jobs/{}"

html_parser_prompt = '''
You are html parser.you here to retrieve the text information from the given html page if present.
Provide the answer in below python parsable json string:
{"title": job title as string else empty string, "level": job level as string else empty string,"location": job location as string else empty string,"description": about job description as string else empty string,"salary": salary range as string else empty string,"key_qualifications": job Minimum qualifications as string or empty string, "preferred_qualifications": job preferred qualifications as string else empty string, "responsibilities": job Responsibilities as string else empty string}
Do not ask for other information. If you don't get information, return answer with empty json.
'''

def load_visited_files(dir=meta_json_jobs_dir)-> set:
    visited:set = set()
    for file in os.listdir(dir):
        jid = file.replace('.json','')
        visited.add(jid)
    return visited

html_parser_agent = ConversableAgent(
    name="html_parser",
    system_message=html_parser_prompt,
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    max_consecutive_auto_reply=1
)

In [3]:
cnt = 0
visited:set = load_visited_files()

for html_file_name in os.listdir(meta_html_jobs_dir):
    google_job_html_file_path = os.path.join(meta_html_jobs_dir, html_file_name)
    page_str=''
    job_id = html_file_name.replace('.html','')

    # if job_id in visited:
    #     print('visited: '+job_id)
    #     continue
    
    with open(google_job_html_file_path, 'r') as file:
        page_str = file.read()
    
    soup = BeautifulSoup(page_str, "html.parser")
    results = soup.find_all("div", attrs={'id': 'careersContentContainer'})
    #print(str(results[0]))
    html_str=str(results[0])

    reply = html_parser_agent.generate_reply(
        messages=[{"content": html_str, "role": "user"}]
    )

    if reply:
        message = reply
        message = message.replace("```json",'')
        message = message.replace("```",'')
        job_json = json.loads(message)
        job_json['company'] = 'meta'
        job_json['url'] = meta_job_url.format(job_id)
        job_json_str = json.dumps(job_json)
        print(job_json_str)
        # json_path = os.path.join(google_json_jobs_dir, job_id + '.json')
        # with open(json_path, 'w') as file:
        #     file.write(job_json_str)
        # print('written: ' + json_path)
    if cnt > 2:
        break 
    cnt+=1

{"title": "Localization Program Manager, Reality Labs Testing", "level": "", "location": "Bellevue, WA | Menlo Park, CA | Seattle, WA | New York, NY", "description": "Languages are key to our mission of bringing the world closer together. The Meta family of products is currently available in 100+ languages and we are adding many more. Over 70% of people use our products in a language other than English, and ~90% are outside of North America. As a Localization Program Manager for Reality Labs Testing, you will partner with stakeholders across Reality Labs and the rest of internationalization teams to ensure extended reality experiences on our products feel local in each market.", "salary": "US$129,000/year to US$185,000/year + bonus + equity + benefits", "key_qualifications": "5+ years experience in program management, including experience shipping large-scale global products. Track record of gaining alignment with cross-org stakeholders with competing priorities and goals. Experience u